# 숫자 인식 모델 
___
* Python : 3.10.9
* cv2 : 4.8.0

In [62]:
import numpy as np
import cv2

# 경로에 한글이 있는경우 로드되지 않는 문제를 해결하고자 한글이 있어도 로드가능하도록 함수를 생성
def hangulFilePathImageRead ( filePath ) :

    stream = open( filePath.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)

    return cv2.imdecode(numpyArray , cv2.IMREAD_UNCHANGED)


### 파일 '0'에 있는 데이터들 전부 배열로 바꿔 주었다.

In [69]:
import os

# 디렉토리 경로
directory_path = r'./data/번호판/train/0'

# 디렉토리 내의 파일 목록을 가져옵니다
file_list = os.listdir(directory_path)

# 확장자가 '.jpg'인 이미지 파일 이름만 추출합니다
image_file_list = [file for file in file_list if file.lower().endswith('.jpg')]

# 최대 300개의 파일만 가져옵니다
max_files_to_get = 300
image_file_list = image_file_list[:max_files_to_get]



In [70]:
image_file_list[0]

'01가0865.jpg'

In [65]:
train_input = []
train_label = []

In [71]:


for img_i in image_file_list:
    file_path = f'{directory_path}/{img_i}'
    img_ori = hangulFilePathImageRead(file_path)
    # 이미지를 28*28 사이즈로 변경
    img_ori = cv2.resize(img_ori, (28, 28))
    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    # 가우시안 블러를 통해 잡음을 제거
    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)
    img_blur_thresh = cv2.adaptiveThreshold(
        img_blurred,
        maxValue = 1,  # 값을 이진화 (0,1)
        adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType = cv2.THRESH_BINARY_INV,
        blockSize = 19,
        C=9
    )

    train_input.append(img_blur_thresh)
    train_label.append(0)  # 해당 이미지의 레이블을 0으로 지정


In [72]:
train_input[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 

In [73]:
train_label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [74]:
image_shape = train_input[0].shape
print(image_shape)

image_shape = train_input[1].shape
print(image_shape)

(28, 28)
(28, 28)


## CNN모델 설계

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D

In [76]:
# Yann LeCun이 개발한 모델 구조 차용

LeNet = Sequential([InputLayer(input_shape=(28,28,1)), 
                    ZeroPadding2D((2,2)),
                    Conv2D(6,5, activation="tanh"),
                    AveragePooling2D(strides=2),
                    Conv2D(16,5,activation="tanh"),
                    AveragePooling2D(strides=2),
                    Conv2D(120,5, activation="tanh"),
                    Flatten(),
                    Dense(84,activation="tanh"),
                    Dense(10,activation="softmax")])

In [77]:
LeNet.compile(optimizer="SGD",
              loss="sparse_categorical_crossentropy",
              metrics="accuracy")

In [78]:
LeNet.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_5 (ZeroPadd  (None, 32, 32, 1)         0         
 ing2D)                                                          
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_4 (Avera  (None, 14, 14, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_16 (Conv2D)          (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_5 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                      

In [79]:
import numpy as np
import tensorflow as tf  # 혹시 모델을 TensorFlow로 정의하고 있다면 필요한 라이브러리

# 데이터를 NumPy 배열로 변환
train_input = np.array(train_input, dtype=np.float32)
train_label = np.array(train_label, dtype=np.int32)



dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))

In [80]:
LeNet.fit(train_input, train_label, epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 4ms/step - loss: 2.2460 - accuracy: 0.1892
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 1.9429 - accuracy: 0.5946
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 1.6288 - accuracy: 0.5946
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 1.3276 - accuracy: 0.5946
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 1.0580 - accuracy: 0.5946
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.8406 - accuracy: 0.9459
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6759 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.5591 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.4757 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 4ms/step - loss: 0.4103 - accuracy: 1.0000


In [85]:


file_path = './data/번호판/test/1/1.jpg'
img_ori = hangulFilePathImageRead(file_path)
img_ori = cv2.resize(img_ori, (28, 28))

height, width, channel = img_ori.shape

gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)

img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)
img_blur_thresh = cv2.adaptiveThreshold(
    img_blurred,
    maxValue=1,  # 변경된 부분: maxValue를 1로 설정
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=19,
    C=9
)

# 이미지 데이터를 0과 1로 표현
image_data = img_blur_thresh



In [86]:
predictions = LeNet.predict(np.expand_dims(image_data, axis=0))

1/1 [==============================] - 0s 13ms/step


In [87]:
# 클래스 레이블 또는 확률 출력

predicted_class = np.argmax(predictions)
print("Predicted Class:", predicted_class)

Predicted Class: 1
